## Stack

In [4]:
class Empty(Exception):
    """Error attempting to access an element from an empty container."""
    pass

In [3]:
class ArrayStack:
    """LIFO Stack implementation using a Python list as underlying storage."""
    
    def __init__(self):
        """Create an empty stack."""
        self._data = []
        
    def __len__(self):
        """Return the number of elements in the stack."""
        return len(self._data)
    
    def is_empty(self):
        """Return True if the stack is empty."""
        return len(self._data) == 0
    
    def push(self, e):
        """Add element e to the top of the stack."""
        self._data.append(e)
        
    def pop(self):
        """Remove and return the element from the top of the stack (i.e., LIFO).
        Raise Empty exception if the stack is empty.
        """
        if self.is_empty():
            raise Empty('Stack is Empty')
        return self._data.pop() 
    
    def top(self):
        """Return (but do not remove) the element at the top of the stack.
        
        Raise Empty exception if the stack is empty.
        """
        if self.is_empty():
            raise Empty('Stack is Empty')
        return self._data[-1]
    
if __name__ == '__main__':
    s = ArrayStack()
    s.push(5)
    s.push(6)
    print(len(s))
    print(s.top())
    print(s.is_empty())
    print(s.pop())
    print(s.is_empty())
    print(s.pop())
    print(s.is_empty())
    s.push(7)
    s.push(8)
    print(len(s))

2
6
False
6
False
5
True
2


## Reversing Data Using a Stack

In [4]:
def reverse_file(filename):
    """Overwrite given file with its contents line-by-line reversed."""
    
    S = ArrayStack()
    original = open(filename)
    for line in original:
        S.push(line.rstrip('\n')) # we will re-insert newlines when writing
    original.close()

    # now we overwrite with contents in LIFO order
    output = open(filename,'w') # reopening file overwrites original
    while not S.is_empty():
        output.write(S.pop() +'\n') # re-insert newline characters
    output.close()
    

    op = open(filename)
    for line in op:
        print(line)
    output.close()

    
filename = 'sample.txt'
print(reverse_file(filename))

Shanta

bk

honnalli

bijapura

sjsu

None


## Matching Parentheses and HTML Tags

In [5]:
def is_matched(expr):
    """Return True if all delimiters are properly match; False otherwise."""
    lefty ='({[' # opening delimiters
    righty = ')}]' # respective closing delims
    S = ArrayStack()
    for c in expr:
        if c in lefty:
            S.push(c) # push left delimiter on stack
        elif c in righty:
            if S.is_empty():
                return False # nothing to match with
            if righty.index(c) != lefty.index(S.pop( )):
                return False # mismatched
    return S.is_empty() # were all symbols matched?

In [6]:
expr = '[()]'
print(is_matched(expr)) # O(n)
expr = '[())]'
print(is_matched(expr))

True
False


## Matching Tags in a Markup Language

In [7]:
def is_matched_html(raw):
    """Return True if all HTML tags are properly match; False otherwise."""
    S = ArrayStack()
    j = raw.find('<') # find first ’<’ character (if any)
    while j != -1:
        k = raw.find('>', j+1) # find next ’>’ character
        if k != -1:
            return False # invalid tag
        tag = raw[j+1:k] # strip away < >
        if not tag.startswith('/'): # this is opening tag
            S.push(tag)
        else: # this is closing tag
            if S.is_empty():
                return False # nothing to match with
            if tag[1:] != S.pop():
                return False # mismatched delimiter
        j = raw.find('<', k+1) # find next ’<’ character (if any)
    return S.is_empty()

In [8]:
raw = '''<body>
<center>
<h1> The Little Boat </h1>
</center>
<p> The storm tossed the little
boat like a cheap sneaker in an
old washing machine. The three
drunken fishermen were used to
such treatment, of course, but
not the tree salesman, who even as
a stowaway now felt that he
had overpaid for the voyage. </p>
<ol>
<li> Will the salesman die? </li>
<li> What color is the boat? </li>
<li> And what about Naomi? </li>
</ol>
</body>'''

print(is_matched_html(raw))

False


## Queues

In [15]:
class ArrayQueue:
    """FIFO queue implementation using a Python list as underlying storage.
    
        _data: is a reference to a list instance with a fixed capacity.
        _size: is an integer representing the current number of elements stored
               in the queue (as opposed to the length of the data list).
        _front: is an integer that represents the index within data of the first
                element of the queue (assuming the queue is not empty).
                
        All operations are O(1) time complexity
        O(n) space complexity, where n is the current number of elements in the queue
    """
    DEFAULT_CAPACITY = 10 # moderate capacity for all new queues
    
    def __init__(self):
        """Create an empty queue."""
        self._data = [None] * ArrayQueue.DEFAULT_CAPACITY
        self._size = 0
        self._front = 0
        
    def __len__(self):
        """Return the number of elements in the queue."""
        return self._size
    
    def is_empty(self):
        """Return True if the queue is empty."""
        return self._size == 0
    
    def first(self):
        """Return (but do not remove) the element at the front of the queue.
        
        Raise Empty exception if the queue is empty.
        """
        if self.is_empty():
            raise Empty('Queue is empty')
        return self._data[self._front]
    
    def dequeue(self):
        """Remove and return the first element of the queue (i.e., FIFO).
        
        Raise Empty exception if the queue is empty.
        """
        if self.is_empty():
            raise Empty('Queue is empty')
        answer = self._data[self._front]
        self._data[self._front] = None  #Help garbage collection
        self._front = (self._front + 1) % len(self._data)
        self._size -= 1
        if 0 < self._size < len(self._data) // 4:
            self._resize(len(self._data) // 2)
        return answer
    
    def enqueue(self,e):
        """Add an element to the back of queue."""
        if self._size == len(self._data):
            self._resize(2 * len(self._data)) # # double the array size
        avail = (self._front + self._size) % len(self._data)
        self._data[avail] = e
        self._size += 1
    
    def _resize(self, cap):
        """Resize to a new list of capacity >= len(self)."""
        old = self._data                 # keep track of existing list
        self._data = [None] * cap        # allocate list with new capacity
        walk = self._front
        for k in range(self._size):      # only consider existing elements
            self._data[k] = old[walk]    # intentionally shift indices
            walk = (1 + walk) % len(old) # use old size as modulus
        self._front = 0                  # front has been realigned
            

In [21]:
Q = ArrayQueue()
Q.enqueue(10)
Q.first()
Q.enqueue('sbk')
Q.is_empty()
Q.dequeue()
Q.first()
len(Q)



1